In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import time
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Reading the train classes that contains the image names and tags from the directory

train_classes = pd.read_csv('/content/train_v2.csv')

train_classes.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [ ]:

labels = set()
def splitting_tags(tags):
    '''
    Takes in tags column, splits the tags and store as a set
    '''
    [labels.add(tag) for tag in tags.split()]

# Create a copy of train_classes
train_classes1 = train_classes.copy()
train_classes1['tags'].apply(splitting_tags)
labels = list(labels)
print(labels)

['slash_burn', 'cultivation', 'conventional_mine', 'water', 'blooming', 'blow_down', 'primary', 'road', 'cloudy', 'artisinal_mine', 'bare_ground', 'clear', 'partly_cloudy', 'selective_logging', 'agriculture', 'habitation', 'haze']


In [ ]:
#confirm that the length of the dataframe is the same as the shape
assert len(train_classes1['image_name'].unique()) == train_classes1.shape[0]

In [ ]:
##One hot encoding is performed on the labels in train classes

for tag in labels:
    train_classes1[tag] = train_classes1['tags'].apply(lambda x: 1 if tag in x.split() else 0)

## adding .jpg extension to the column image_name so as to have same name format as the image files
train_classes1['image_name'] = train_classes1['image_name'].apply(lambda x: '{}.jpg'.format(x))
train_classes1.head()

,image_name,tags,slash_burn,cultivation,conventional_mine,water,blooming,blow_down,primary,road,cloudy,artisinal_mine,bare_ground,clear,partly_cloudy,selective_logging,agriculture,habitation,haze
0,train_0.jpg,haze primary,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
1,train_1.jpg,agriculture clear primary water,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0
2,train_2.jpg,clear primary,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
3,train_3.jpg,clear primary,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
4,train_4.jpg,agriculture clear habitation primary road,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,1,0


In [ ]:
#importing libraries for training
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, Flatten
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Defining the columns,i.e the labels that were newly added to the train_classes via hot encoding.
columns = list(train_classes1.columns[2:])

In [ ]:
columns

['slash_burn',
 'cultivation',
 'conventional_mine',
 'water',
 'blooming',
 'blow_down',
 'primary',
 'road',
 'cloudy',
 'artisinal_mine',
 'bare_ground',
 'clear',
 'partly_cloudy',
 'selective_logging',
 'agriculture',
 'habitation',
 'haze']

In [ ]:
def fbeta(y_true, y_pred, beta = 2, epsilon = 1e-4):
    '''
    Set y_true and y_pred

    Args:
        y_true: correct target values
        Y_pred: predicted values returned by the classifer
        beta = 2
        epsilon= 1e-4

    Returns:
        fbeta score
    '''

    beta_squared = beta**2

    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(tf.greater(tf.cast(y_pred, tf.float32), tf.constant(0.5)), tf.float32)

    tp = tf.reduce_sum(y_true * y_pred, axis = 1)
    fp = tf.reduce_sum(y_pred, axis = 1) - tp
    fn = tf.reduce_sum(y_true, axis = 1) - tp

    precision = tp/(tp+fp+epsilon)
    recall = tp/(tp+fn+epsilon)

    fb = (1+beta_squared)*precision*recall / (beta_squared*precision+recall+epsilon)
    return fb


In [ ]:
def multi_label_acc(y_true, y_pred, epsilon = 1e-4):
    '''
    Retuns accuracy value for multi_label classification

    Set y_true and y_pred

    Args:
        y_true: correct target values
        Y_pred: predicted values returned by the classifer
        epsilon= 1e-4

    Returns:
        Accuracy score
    '''
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(tf.greater(tf.cast(y_pred, tf.float32), tf.constant(0.5)), tf.float32)

    tp = tf.reduce_sum(y_true * y_pred, axis = 1)
    fp = tf.reduce_sum(y_pred, axis = 1) - tp
    fn = tf.reduce_sum(y_true, axis = 1) - tp

    y_true = tf.cast(y_true, tf.bool)
    y_pred = tf.cast(y_pred, tf.bool)

    tn = tf.reduce_sum(tf.cast(tf.logical_not(y_true), tf.float32)
                       * tf.cast(tf.logical_not(y_pred), tf.float32), axis = 1)

    return (tp+tn)/(tp+tn+fp+fn+epsilon)

In [ ]:
#defining our model
def build_model():
    model = Sequential()
    model.add(BatchNormalization(input_shape=(128, 128, 3)))
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(17, activation='sigmoid'))

    opt = Adam(lr=1e-4)

    # We need binary here, since categorical_crossentropy l1 norms the output before calculating loss.
    model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=[multi_label_acc, fbeta])

    return model

In [ ]:
#modelcheckpoint is set to monitor the model using validation fbeta score and save the best only
save_best_check_point = ModelCheckpoint(filepath = 'best_model.hdf5',
                                        monitor = 'val_fbeta',
                                        mode = 'max',
                                        save_best_only = True,
                                        save_weights_only = True)

In [ ]:
#initializing imagedatagenerator with a validation split of 0.2
train_image_gen = ImageDataGenerator(rescale = 1/255, validation_split = 0.2)

#generating train data generator which is 80% of the train dataset
#note that a generator contains both features and target of the data
train_generator = train_image_gen.flow_from_dataframe(dataframe=train_classes1,
                                                directory ="/content/train-jpg.tar",
                                                x_col="image_name", y_col=columns, subset="training",
                                                batch_size=16,seed=2021, shuffle=True,
                                                class_mode="raw", target_size=(128,128))

#generating validation data which is expected to be 20% of the train dataset since validation split is 0.2
val_generator = train_image_gen.flow_from_dataframe(dataframe=train_classes1,
                                                directory ="/content/train-jpg.tar",
                                                x_col="image_name", y_col=columns, subset="validation",
                                                batch_size=16,seed=2021, shuffle=True,
                                                class_mode="raw", target_size=(128,128))


Found 0 validated image filenames.
Found 0 validated image filenames.


In [ ]:
#setting up step size for training and validation image data
step_train_size = int(np.ceil(train_generator.samples / train_generator.batch_size))
step_val_size = int(np.ceil(val_generator.samples / val_generator.batch_size))

In [ ]:
#initialize the model
model1 = build_model()

In [ ]:
# Preview the model architecture
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 128, 128, 3)      12        
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 126, 126, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 63, 63, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 63, 63, 64)        1